In [ ]:
from flask import *
import pandas as pd
import numpy as np
import joblib
import re
import nltk
from nltk.corpus import stopwords
import string
import requests
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword = set(stopwords.words('english'))

app = Flask(__name__)
bad_words = pd.read_csv('bad-words.csv')
cv = joblib.load('vector.joblib')
clf = joblib.load('model.joblib')

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

def predict_with_asterisks(user_input):
    t = clean(user_input)
    t = cv.transform([t])
    output = clf.predict(t)
    if output == 'No hate speech':
        return user_input
    else:
        for word in bad_words['jigaboo']:
            user_input = user_input.replace(word, '*' * len(word))
        return user_input

def translate(source_text,source):
    source_lang = source
    target_lang = 'en'
    url = f"https://translate.googleapis.com/translate_a/single?client=gtx&sl={source_lang}&tl={target_lang}&dt=t&q={source_text}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        result_text = data[0][0][0]
        return result_text
    else:
        return 'could not translate'
    
def ttranslate(source_text,source):
    source_lang = 'en'
    target_lang = source
    url = f"https://translate.googleapis.com/translate_a/single?client=gtx&sl={source_lang}&tl={target_lang}&dt=t&q={source_text}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        result_text = data[0][0][0]
        return result_text
    else:
        return 'could not translate'
    
@app.route('/')
def hello():
    return render_template('index.html',pred='')
@app.route('/', methods=['POST'])
def predict():
    fea = [str(x) for x in request.form.values()]
    s=translate(fea[1],fea[0])
    s=predict_with_asterisks(s)
    s=ttranslate(s,fea[0])
    return render_template('index.html',pred=s)
    
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Apr/2024 15:41:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 15:42:10] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 15:42:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 15:43:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 15:43:42] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 15:43:46] "POST / HTTP/1.1" 200 -
